<a href="https://colab.research.google.com/github/Shubhamd13/NLP/blob/main/Hw1_Skip_Gram_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>





# HW1: Skip-Gram Model From Scratch

In [ ]:
%matplotlib inline
# put your student name here, you will need to train the model that prints out your name in each loss
# without the name, you will not be able to get points in train part
STUDENT_NAME = ""

This section imports necessary Python libraries such as NumPy and the random module. These libraries are essential for generating random training data and managing numerical computations.

In [ ]:
import numpy as np
import random
np.random.seed(0)
random.seed(0)

### 1.Create Corpus

In this section, you should first define a custom vocabulary composed of universities in Southern California (SoCal) and Northern California (NorCal). It generates a synthetic corpus by randomly pairing elements within each regional list to simulate word-context relationships.

In [ ]:
SoCal = ["UCR", "UCLA", "UCSD", "UCI", "UCSB"]
NorCal = ["UCB", "UCSC", "UCD", "UCSF", "UCM"]
vocab = SoCal + NorCal

corpus = []

# Create Corpus
for i in range(20):
  random.shuffle(SoCal)
  corpus.append((SoCal[0], SoCal[1]))

for i in range(20):
  random.shuffle(NorCal)
  corpus.append((NorCal[0], NorCal[1]))

word_to_index = {word: i for i, word in enumerate(vocab)}
index_to_word = {i: word for i, word in enumerate(vocab)}

vocab_size = len(vocab)

embedding_dim = 2
learning_rate = 0.02
num_neg_samples = 2

print(corpus)

[('UCSD', 'UCLA'), ('UCSD', 'UCR'), ('UCR', 'UCSD'), ('UCI', 'UCR'), ('UCR', 'UCSB'), ('UCSB', 'UCLA'), ('UCLA', 'UCSB'), ('UCI', 'UCLA'), ('UCI', 'UCSB'), ('UCLA', 'UCSB'), ('UCR', 'UCI'), ('UCI', 'UCLA'), ('UCI', 'UCR'), ('UCSB', 'UCSD'), ('UCI', 'UCSD'), ('UCSD', 'UCSB'), ('UCLA', 'UCSD'), ('UCSB', 'UCLA'), ('UCR', 'UCI'), ('UCLA', 'UCSD'), ('UCSF', 'UCD'), ('UCB', 'UCD'), ('UCM', 'UCSC'), ('UCM', 'UCD'), ('UCSF', 'UCB'), ('UCB', 'UCSC'), ('UCSC', 'UCD'), ('UCSC', 'UCB'), ('UCB', 'UCSF'), ('UCSF', 'UCD'), ('UCM', 'UCSC'), ('UCSC', 'UCD'), ('UCM', 'UCSC'), ('UCD', 'UCM'), ('UCM', 'UCB'), ('UCB', 'UCD'), ('UCM', 'UCB'), ('UCB', 'UCD'), ('UCB', 'UCSC'), ('UCSF', 'UCB')]


### 2.Define Visualization Function

Implement plotting utilities for visualizing training loss, embedding space, and cosine similarity between word vectors. (No Need to Change.)


In [ ]:
import matplotlib.pyplot as plt

# Plot loss over epochs
def plot_loss_over_epochs(loss_values, epochs):
  plt.figure(figsize=(6, 4))
  plt.plot(range(1, epochs + 1), loss_values, marker='o')
  plt.ticklabel_format(style='plain', axis='y', useOffset=False)
  plt.xlabel('Epoch')
  plt.ylabel('Average Loss')
  plt.title('Loss over Epochs')
  plt.show()

# Plot Embedding
def plot_embedding(matrix, vocab):
  plt.figure(figsize=(6, 4))
  x = matrix[:, 0]
  y = matrix[:, 1]
  plt.scatter(x, y)
  for i, word in enumerate(vocab):
    plt.annotate(word, xy=(x[i], y[i]))

  plt.title("Embedding Visualization")
  plt.show()

# Plot Similary score
def plot_cosine_similarity(matrix, vocab):

  def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

  similarities = np.array([
      cosine_similarity(matrix[0], emb) for emb in matrix
  ])

  plt.figure(figsize=(6, 4))
  plt.bar(range(len(similarities)), similarities)
  plt.xticks(range(len(similarities)), vocab)
  plt.ylim(-1.1, 1.1)
  plt.ylabel("Cosine Similarity")
  plt.title("Cosine Similarity to UCR")
  plt.grid(axis='y')

  plt.show()

### 3.Define trainable parameters (10pt)

Use Numpy to initialize two trainable embedding matrices — one for context words and one for target words — with small random values.

Here is the size for each Matrix:

1. Context Matrix:   [vocab_size, embedding_dim]
2. Target Matrix:  [vocab_size, embedding_dim]

In [ ]:
def initialize_parameters(vocab_size, embedding_dim):
  # Context Matrix
  ####! add your code here
  W1 =
  ####

  # Target Matrix
  ####! add your code here
  W2 =
  ####

  return W1, W2

# visualization
W1, W2 = initialize_parameters(vocab_size, embedding_dim)
plot_embedding(W2, vocab)
plot_cosine_similarity(W2, vocab)

### 4.Define Activation Function (5pt)
Implement the sigmoid function used in the computation of prediction probabilities and gradients.


Sigmoid Function: $\sigma(z) = \frac{1}{ 1 + exp(-z)}$

In [ ]:
#! define sigmoid function
def sigmoid(z):
  z = np.clip(z, -100, 100)
  ####! add your code here
  return

### 5.Define Negative Sampling (15pt)

Randomly select a set of negative word samples for each training instance, ensuring they differ from the current positive pair.

By using negative sampling, we can improve computational efficiency by approximating the full softmax over the vocabulary.


In [ ]:
def negative_sampling(word_to_index, target_index, context_index, num_neg_samples):
  """
  word_to_index: vocabulary.
  target_index: target word index.
  num_neg_samples: numbers of negative samples.
  """
  vocab_indices = list(word_to_index.values())
  # remove context and target tokens.
  ####! add your code here

  ####
  # randomly choose negative words.
  ####! add your code here
  neg_samples =
  ####

  return neg_samples


# test Negative Sampling
neg_samples = negative_sampling(word_to_index, word_to_index["UCR"], word_to_index["UCLA"], num_neg_samples)
print('Negative Words:')
for sample in neg_samples:
  print(index_to_word[sample])

### 6.Forward (20pt)
Compute the positive and negative prediction scores and calculate the loss using the negative sampling technique.

Loss Function:
$L_{CE}=-\bigg[log\sigma(c_{pos}\cdot w) + \sum_{i=1}^{k}log\sigma(-c_{neg_i}\cdot w)\bigg]$

In [ ]:
def forward_negative_sampling(context_word_index, target_word_index, neg_samples, W1, W2):
  # obtain context vector
  h_context = W1[context_word_index]
  # obtain negative sample vector
  ####! Add your code here
  h_neg =
  ####

  # ompute positive prediction and loss.
  u_target = np.dot(h_context, W2[target_word_index])
  ####! Add your code here
  pos_loss =
  ####

  # compute negative prediction and loss.
  ####! Add your code here
  u_neg_samples =
  neg_loss =
  ####

  # total loss
  loss = pos_loss + neg_loss
  return u_target, u_neg_samples, loss

### 7.Backward (20pt)
Update model parameters using gradients derived from the loss function for both positive and negative samples.

Loss Function:

1. $c^{t+1}_{pos} = c^t_{pos} - \eta [\sigma(c^t_{pos}\cdot w^t)-1]w^t$

2. $c^{t+1}_{neg} = c^t_{neg} - \eta [\sigma(c^t_{neg}\cdot w^t)]w^t$

3. $w^{t+1} = w^t - \eta \bigg[ [\sigma (c_{pos} \cdot w^t - 1)]c_{pos} + \sum_{i=1}^{k}[\sigma (c_{neg_i}\cdot w^t)] c_{neg_i}\bigg]$

In [ ]:
def backward_negative_sampling(
      u_target,
      u_neg_samples,
      context_word_index,
      target_word_index,
      neg_samples_idx,
      W1,
      W2,
      learning_rate):
  """
  u_target: positive prediction.
  u_negative_samples: negative predictions.
  target_word_index: positive prediction word.
  neg_samples_idx: negative prediction words.
  W1: Embedding matrix.
  W2: Projection matrix.
  """
  # positive gradient.
  grad_pos = sigmoid(u_target) - 1
  W2[target_word_index] -= learning_rate * grad_pos * W1[context_word_index]
  ####! Add your code here
  W1[context_word_index] -=
  ####

  # negative gradient.
  ####! Add your code here
  grad_neg =
  ####

  for i, neg_index in enumerate(neg_samples_idx):
      ####! Add your code here
      W2[target_word_index] -=
      W1[neg_index] -=
      ####

### 8.Train Skip-Gram Model (20pt)

Loop through the corpus over multiple epochs, applying forward and backward passes to learn word embeddings. You can train the model to produce meaningful word embeddings that capture contextual similarities.


In [ ]:
def train_skip_gram_negative_sampling(corpus, word_to_index, vocab_size, embedding_dim, learning_rate, epochs, num_neg_samples):
  """
  corpus: Training corpus.
  word_to_index: vocabulary.
  vocab_size: Size of vocabulary.
  embedding_dim: Dimension of Embedding matrix.
  learning_rate: learning rate for optimization.
  epochs: Training epochs.
  num_neg_samples: numbers of negative samples used during training.
  """
  # init parameters
  W1, W2 = initialize_parameters(vocab_size, embedding_dim)
  total_loss = []
  for epoch in range(epochs):
      loss_epoch = 0
      for context_word, target_word in corpus:

          # convert to index.
          context_word_index = word_to_index[context_word]
          target_word_index =

          # Obtain negative samples.
          ####! Add your code here
          neg_samples_index =
          ####

          # Forward
          ####! Add your code here
          u_target, u_neg_samples, loss =
          ####
          loss_epoch += loss

          # Backward
          ####! Add your code here

          ####

      avg_loss = loss_epoch / len(corpus)
      print(f"{STUDENT_NAME} Epoch {epoch+1}/{epochs}, Loss: {avg_loss}")
      total_loss.append(avg_loss)

  return W1, W2, total_loss

### 9.Run Skip-Gram Model

Executes the training process and visualizes the final word embeddings.




In [ ]:
epochs = 10
W1, W2, total_loss = train_skip_gram_negative_sampling(corpus, word_to_index, vocab_size, embedding_dim, learning_rate, epochs, num_neg_samples)
plot_embedding(W1, vocab)
plot_loss_over_epochs(total_loss, epochs)
plot_cosine_similarity(W2, vocab)

In [ ]:

epochs = 100
W1, W2, total_loss = train_skip_gram_negative_sampling(corpus, word_to_index, vocab_size, embedding_dim, learning_rate, epochs, num_neg_samples)
plot_embedding(W1, vocab)
plot_loss_over_epochs(total_loss, epochs)
plot_cosine_similarity(W2, vocab)

In [ ]:
epochs = 200
W1, W2, total_loss = train_skip_gram_negative_sampling(corpus, word_to_index, vocab_size, embedding_dim, learning_rate, epochs, num_neg_samples)
plot_embedding(W1, vocab)
plot_loss_over_epochs(total_loss, epochs)
plot_cosine_similarity(W2, vocab)

### 10.Use Pre-trained Word2Vector Model (10pt)

In this section, you should use pretrained GloVe word embeddings from the gensim library to compute the semantic similarity between sentences.



In [ ]:
import numpy as np
from nltk.tokenize import word_tokenize
import gensim.downloader
import nltk
import string
from nltk.corpus import stopwords

nltk.download('punkt_tab')
nltk.download('stopwords')

# List all pretrained embedding models
print(list(gensim.downloader.info()['models'].keys()))

# Download the "glove-twitter-25" embeddings
models = gensim.downloader.load('glove-twitter-25')

ModuleNotFoundError: No module named 'gensim'

In [ ]:
def review_to_vector(review_text):
    # Tokenize and clean the review
    words = word_tokenize(review_text.lower())
    words = [word for word in words if word not in string.punctuation]
    words = [word for word in words if word not in stopwords.words('english')]

    # Remove words not in vocabulary
    ####! Add your code here

    ####
    if not valid_words:
        return np.zeros(models.vector_size)

    # Average word vectors
    ####! Add your code here
    avg_vec =
    ####
    return avg_vev

def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [ ]:
# Three Sentences.
sent1 = "The cat is sleeping on the sofa."
sent2 = "A cat is napping on the couch."
sent3 = "The spaceship landed on Mars after a six-month journey."

# Convert reviews to vectors
vec1 = review_to_vector(sent1)
vec2 = review_to_vector(sent2)
vec3 = review_to_vector(sent3)

# Calculate similarities
sim1 = cosine_similarity(vec1, vec2)
sim2 = cosine_similarity(vec1, vec3)

print(f"sent1-sent2 similarity: {sim1:.4f}")
print(f"sent1-sent3 similarity: {sim2:.4f}")